# From Llama to Alpaca: Finetunning an LLM with Weights & Biases
In this notebook, we will use an updated version of the Alpaca dataset that, instead of davinci-003 (GPT3) generations uses GPT4 to get an even better instruction dataset!

In [ ]:
!pip install -q wandb transformers

Let's grab the Alpaca (GPT-4 curated instructions and outputs) dataset:

In [ ]:
!wget https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json

In [ ]:
import json

dataset_file = "alpaca_gpt4_data.json"

with open(dataset_file, "r") as f:
    alpaca = json.load(f)

So the dataset has instruction and outputs. The model is trained to predict the next token, so one option would be just to concat both, and train on that. We ideally format the prompt in a way that we make explicit where is the input and output. Let's log the dataset to W&B so we keep everything organised

In [ ]:
import wandb

# log to wandb
with wandb.init(project="alpaca_ft"):
    at = wandb.Artifact(
        name="alpaca_gpt4", 
        type="dataset",
        description="A GPT4 generated Alpaca like dataset for instruction finetunning",
        metadata={"url":"https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data"},
    )
    at.add_file(dataset_file)

    # log as a table
    table = wandb.Table(columns=list(alpaca[0].keys()))
    for row in alpaca:
        table.add_data(*row.values())
    wandb.log({"alpaca_gpt4_table": table})

### Train/Eval Split

In [ ]:
import random

seed = 42

random.seed(seed)
random.shuffle(alpaca)  # this could also be a parameter

In [ ]:
train_dataset = alpaca[:-1000]
eval_dataset = alpaca[-1000:]

In [ ]:
import pandas as pd

train_df = pd.DataFrame(train_dataset)
eval_df = pd.DataFrame(eval_dataset)

train_table = wandb.Table(dataframe=train_df)
eval_table  = wandb.Table(dataframe=eval_df)

train_df.to_json("alpaca_gpt4_train.jsonl", orient='records', lines=True)
eval_df.to_json("alpaca_gpt4_eval.jsonl", orient='records', lines=True)

with wandb.init(project="alpaca_ft", job_type="split_data"):
    at = wandb.Artifact(
        name="alpaca_gpt4_splitted", 
        type="dataset",
        description="A GPT4 generated Alpaca like dataset for instruction finetunning",
        metadata={"url":"https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data"},
    )
    at.add_file("alpaca_gpt4_train.jsonl")
    at.add_file("alpaca_gpt4_eval.jsonl")
    wandb.log_artifact(at)
    wandb.log({"train_dataset":train_table, "eval_dataset":eval_table})

Let's log the dataset also as a table so we can inspect it on the workspace.

In [ ]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n").format_map(row)

In [ ]:
row = alpaca[0]
print(prompt_no_input(row))

Some other instruction have some context in the `input` variable`

In [ ]:
def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n").format_map(row)

In [ ]:
row = alpaca[232]
print(prompt_input(row))

> But you are leaving the output out!!! Yes, but we can just concat that afterwards. Let's deal with the prompt now, we can add the output later with the right amount of padding.

And the refactored function

In [ ]:
def create_alpaca_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

## Why are we doing all this?

In [ ]:
train_prompts = [create_alpaca_prompt(row) for row in train_dataset]
eval_prompts = [create_alpaca_prompt(row) for row in eval_dataset]

We need to process the targets and add the End Of String token (EOS) to the results. For LLama this is: `"</s>"`

In [ ]:
def pad_eos(ds):
    EOS_TOKEN = "</s>"
    return [f"{row['output']}{EOS_TOKEN}" for row in ds]

In [ ]:
train_outputs = pad_eos(train_dataset)
eval_outputs = pad_eos(eval_dataset)
train_outputs[0]

Cool! but why we have everything separated? Let's sore the "final" version on a variable called `examples`

In [ ]:
train_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(train_prompts, train_outputs)]
eval_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(eval_prompts, eval_outputs)]

This is what the model need to see and learn =)

In [ ]:
print(train_dataset[0]["example"])

## Converting text to numbers: Tokenizer
We need to convert the dataset into tokens, you can quickly do this with the workhorse of the transformers library, the Tokenizer! This function does a lot of heavy lifting besides tokenizing the text. 

- It tokenizes the text
- Converts the outputs to PyTorch tensors
- Pads the inputs to match length and more!

In [ ]:
# import os
# os.environ['HF_TOKEN'] = ''

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_id = 'meta-llama/Llama-2-7b-hf'
#model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_id) # , token=''
tokenizer.pad_token = tokenizer.eos_token

### Packing

We will pack multiple short examples into a longer chunk, so we can train more efficiently!

The main idea here is that the instruction/output samples are short, so let's concatenate a bunch of them together separated by the `EOS` token. We can also pre-tokenize and pre-pack the dataset and make everything faster!  If we define a `max_seq_len = 1024` the code to pack would look something like this:

In [ ]:
max_sequence_len = 1024

def pack(dataset, max_seq_len=max_sequence_len):
    tkds_ids = tokenizer([s["example"] for s in dataset])["input_ids"]
    
    all_token_ids = []
    for tokenized_input in tkds_ids:
        all_token_ids.extend(tokenized_input)# + [tokenizer.eos_token_id])
    
    print(f"Total number of tokens: {len(all_token_ids)}")
    packed_ds = []
    for i in range(0, len(all_token_ids), max_seq_len+1):
        input_ids = all_token_ids[i : i + max_seq_len+1]
        if len(input_ids) == (max_seq_len+1):
            packed_ds.append({"input_ids": input_ids[:-1],
                              "labels": input_ids[1:],
                              "decoded" : tokenizer.decode(input_ids[1:])})  # this shift is not needed if using the model.loss
    return packed_ds

In [ ]:
train_ds_packed = pack(train_dataset)
eval_ds_packed = pack(eval_dataset)

In [ ]:
# It's weird why this technique was suggested. I understand that it makes full us of GPUs but
# doesn't it waste resources because if an instruction example is split in two halves between two sub-examples, how can the start of the second start with some meaninful information apart from the raw leftoever text?

Doing so, we end up with a little more than 11k sequences of lenght 1024. 

### DataLoader
As we are training for completion, the labels (or targets) will be the inputs shifted by one. We are going to train with regular Cross Entropy and predict the next token on this packed dataset.

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

torch.manual_seed(seed)
batch_size = 4  # I have an A100 GPU with 40GB of RAM 😎

train_dataloader = DataLoader(
    train_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator, # we don't need any special collator 😎
)

eval_dataloader = DataLoader(
    eval_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator,
    shuffle=False,
)

It is always a good idea to check how does a batch looks like, you can quickly do this by sampling from the DataLoader

In [ ]:
b = next(iter(train_dataloader))
b

We can alos decode the batch just to be super sure

In [ ]:
tokenizer.decode(b["input_ids"][0])#[:250]

In [ ]:
tokenizer.bos_token

In [ ]:
tokenizer.decode(b["labels"][0])[:250]

## Train

I like storing all my hyperparameters into a `SimpleNamespace`, it's like a dict but with .dot attribute access. Then I can access my batch size by doing config.batch_size instead of config["batch_size"]. 

In [ ]:
from types import SimpleNamespace

gradient_accumulation_steps = 4

config = SimpleNamespace(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',#'',#
    dataset_name="alpaca-gpt4",
    precision="f16",  # bf16, faster and better than fp16, requires new GPUs
    n_freeze=20,  # How many layers we don't train, LLama 7B has 32., tinyllama has 22
    lr=2e-4,
    n_eval_samples=10, # How many samples to generate on validation
    max_seq_len=max_sequence_len, # Lenght of the sequences to pack
    epochs=3,  # we do 3 pasess over the dataset.
    gradient_accumulation_steps=gradient_accumulation_steps,  # evey how many iterations we update the gradients, simulates larger batch sizes
    batch_size=batch_size,  # what my GPU can handle, depends on how many layers are we training  
    log_model=False,  # upload the model to W&B?
    gradient_checkpointing = True,  # saves even more memory
    freeze_embed = True,  # why train this? let's keep them frozen ❄️
    seed=seed,
)

config.total_train_steps = config.epochs * len(train_dataloader) // config.gradient_accumulation_steps

In [ ]:
print(f"We will train for {config.total_train_steps} steps and evaluate every epoch")

We first get a pretrained model with some configuration parameters

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    config.model_id,
    #device_map=0,
    trust_remote_code=True,
    #low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)

model

In [ ]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

params, trainable_params = param_count(model)

Training the full models is expensive, but if you have a GPU that can fit the full model, you can skip this part. Let's just train the last 8 layers of the model (Llama2-7B has 32)

In [ ]:
token_embeds = next(iter(model.parameters()))
print(token_embeds.shape)
token_embeds

In [ ]:
config.n_freeze = len(model.model.layers) - 5
config.n_freeze

In [ ]:
# freeze layers (disable gradients)
for param in model.parameters(): param.requires_grad = False
for param in model.lm_head.parameters(): param.requires_grad = True
for param in model.model.layers[config.n_freeze:].parameters(): param.requires_grad = True

In [ ]:
# Just freeze embeddings for small memory decrease
if config.freeze_embed:
    model.model.embed_tokens.weight.requires_grad_(False)

and you can also use gradient checkpointing to save even more (this makes training slower, how much it will depend on your particular configuration). There is a [nice article](https://huggingface.co/docs/transformers/v4.18.0/en/performance) on the Huggingface website about how to fit large models on memory, I encourage you to check it!


In [ ]:
# save more memory
if config.gradient_checkpointing:
    model.gradient_checkpointing_enable()

In [ ]:
params, trainable_params = param_count(model)

### Optimizer

We setup the standard optimization stuff...

In [ ]:
from transformers import get_cosine_schedule_with_warmup

optim = torch.optim.Adam(model.parameters(), lr=config.lr, betas=(0.9,0.99), eps=1e-5)
scheduler = get_cosine_schedule_with_warmup(
    optim,
    num_training_steps=config.total_train_steps,
    num_warmup_steps=config.total_train_steps // 10,
)

In [ ]:
def loss_fn(x, y):
    "A Flat CrossEntropy" 
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

## Testing during training

We are almost there, let's create a simple function to sample from the model now and then, to visualy see what the models is outputting!
Let's wrap the model.generate method for simplicity. You can grab the defaults sampling parameters from the GenerationConfig and passing the corresponding model_id. This will grab you the defaults for parameters like temperature, top p, etc...


In [ ]:
from types import SimpleNamespace
from transformers import GenerationConfig

gen_config = GenerationConfig.from_pretrained(config.model_id)
test_config = SimpleNamespace(
    max_new_tokens=256,
    gen_config=gen_config)

In [ ]:
def generate(prompt, max_new_tokens=test_config.max_new_tokens, gen_config=gen_config):
    tokenized_prompt = tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
    with torch.inference_mode():
        output = model.generate(tokenized_prompt, 
                            max_new_tokens=max_new_tokens, 
                            generation_config=gen_config)
    return tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)

In [ ]:
model.to('cuda')

In [ ]:
prompt = eval_dataset[3]["prompt"]
print(prompt + generate(prompt, 128))

We can log a Table with those results to the project every X steps

In [ ]:
import wandb
from tqdm.auto import tqdm

def prompt_table(examples, log=False, table_name="predictions"):
    table = wandb.Table(columns=["prompt", "generation", "concat", "output", "max_new_tokens", "temperature", "top_p"])
    for example in tqdm(examples, leave=False):
        prompt, gpt4_output = example["prompt"], example["output"]
        out = generate(prompt, test_config.max_new_tokens, test_config.gen_config)
        table.add_data(prompt, out, prompt+out, gpt4_output, test_config.max_new_tokens, test_config.gen_config.temperature, test_config.gen_config.top_p)
    if log:
        wandb.log({table_name:table})
    return table

def to_gpu(tensor_dict):
    return {k: v.to('cuda') for k, v in tensor_dict.items()}

class Accuracy:
    "A simple Accuracy function compatible with HF models"
    def __init__(self):
        self.count = 0
        self.tp = 0.
    def update(self, logits, labels):
        logits, labels = logits.argmax(dim=-1).view(-1).cpu(), labels.view(-1).cpu()
        tp = (logits == labels).sum()
        self.count += len(logits)
        self.tp += tp
        return tp / len(logits)
    def compute(self):
        return self.tp / self.count

You can also quickly add validation if you feel so, the table can be also created at this step:

In [ ]:
@torch.no_grad()
def validate():
    model.eval();
    eval_acc = Accuracy()
    loss, total_steps = 0., 0
    for step, batch in enumerate(pbar:=tqdm(eval_dataloader, leave=False)):
        pbar.set_description(f"doing validation")
        batch = to_gpu(batch)
        total_steps += 1
        with torch.amp.autocast("cuda", dtype=torch.float16):
            out = model(**batch)
            loss += loss_fn(out.logits, batch["labels"])  # you could use out.loss and not shift the dataset
        eval_acc.update(out.logits, batch["labels"])
    # we log results at the end
    wandb.log({"eval/loss": loss.item() / total_steps,
               "eval/accuracy": eval_acc.compute()})
    prompt_table(eval_dataset[:config.n_eval_samples], log=True)
    model.train();

In [ ]:
from pathlib import Path
def save_model(model, model_name, models_folder="models", log=False):
    """Save the model to wandb as an artifact
    Args:
        model (nn.Module): Model to save.
        model_name (str): Name of the model.
        models_folder (str, optional): Folder to save the model. Defaults to "models".
    """
    model_name = f"{wandb.run.id}_{model_name}"
    file_name = Path(f"{models_folder}/{model_name}")
    file_name.parent.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(file_name, safe_serialization=True)
    # save tokenizer for easy inference
    tokenizer = AutoTokenizer.from_pretrained(model.name_or_path)
    tokenizer.save_pretrained(model_name)
    if log:
        at = wandb.Artifact(model_name, type="model")
        at.add_dir(file_name)
        wandb.log_artifact(at)

Let's define a loop that compute evaluation and logs a Table with model predictions

## The actual Loop
It's actually nothing fancy, and very short! It has:
- Gradient accumulation and gradient scaling
- sampling and model checkpoint saving (this trains very fast, no need to save multiple checkpoints)
- We compute token accuracy, better metric than loss.

In [ ]:
wandb.init(project="alpaca_ft", # the project I am working on
           tags=["baseline","7b"],
           job_type="train",
           config=config) # the Hyperparameters I want to keep track of

# Training
acc = Accuracy()
model.train()
train_step = 0
for epoch in tqdm(range(config.epochs)):
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = to_gpu(batch)
        with torch.amp.autocast("cuda", dtype=torch.float16):
            out = model(**batch)
            loss = loss_fn(out.logits, batch["labels"]) / config.gradient_accumulation_steps  # you could use out.loss and not shift the dataset  
            
            wandb.log({"train/loss": loss.item() * config.gradient_accumulation_steps,
                       "train/accuracy": acc.update(out.logits, batch["labels"]),
                       "train/learning_rate": scheduler.get_last_lr()[0],
                       "train/global_step": train_step})
            
        if step%config.gradient_accumulation_steps == 0:
            # we can log the metrics to W&B
            optim.step()
            scheduler.step()
            optim.zero_grad(set_to_none=True)
            train_step += 1
    validate()    
    break

In [ ]:
# we save the model checkpoint at the end
save_model(model, model_name=config.model_id.replace("/", "_"), models_folder="models/", log=config.log_model)
wandb.finish()

This trains in around 60 minutes on an A100. 

## Full Eval Dataset evaluation

Let's log a table with model predictions on the eval_dataset (or at least the 250 first samples)

In [ ]:
with wandb.init(project="alpaca_ft", # the project I am working on
           job_type="eval",
           config=config): # the Hyperparameters I want to keep track of
    model.eval();
    prompt_table(eval_dataset[:250], log=True, table_name="eval_predictions")